In [ ]:
import os
import sys
os.environ["THEANO_FLAGS"] = "device=cpu"
sys.path.append(os.getcwd()+"/..")
import lasagne
import theano.tensor as T
import theano
from tasks import check
from scripts import manifold

In [ ]:
v = check(what="notebook", 
          #filename="..//model.pkl", 
          filename="../training//model.pkl",
          dataset='flaticon',
          params=dict(mode='batch'),
          force_w=64,
          force_h=64)
          #dataset="fonts",
          #force_w=28,
          #force_h=28)

In [ ]:
capsule, data, layers, w, h, c = v
print(w, h, c)
print(layers.keys())

In [ ]:
print(layers['input'].shape)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import tile_raster_images
import numpy as np
for name, layer in layers.items():
    #print(name, layer.output_shape)
    #if hasattr(layer, "W"):
    #    print(layer.W.get_value().shape)
    if hasattr(layer, "W"):
        W = layer.W.get_value()
        if len(W.shape)==2:
            nbpixels = c*w*h
            if W.shape[0] == nbpixels:
                W = W.T
            a, b = w, h
        else:
            W = W.reshape((W.shape[0]*W.shape[1], W.shape[2], W.shape[3]))
            a, b = W.shape[1:]
        fig = plt.figure(figsize=(10, 10))
        s = int(np.sqrt(W.shape[0]))
        plt.axis('off')
        img=tile_raster_images(W, (a, b), (s, s))
        plt.imshow(img, cmap="gray", interpolation='none')
        plt.show()

In [ ]:
hid_nonlinearity = 'sigmoid'
out_nonlinearity = 'sigmoid'

def get_deriv(func, x, direct=False):
    if func == 'sigmoid':
        if direct:
            x_ = x
        else:
            x_ = T.nnet.sigmoid(x)
        return x_ * (1 - x_)
    elif func == 'tanh':
        if direct:
            x_  = x
        else:
            x_ = T.tanh(x)
        return 1 - x_ **2

In [ ]:
X = T.tensor4()

Xrec = lasagne.layers.get_output(layers['output'], X)
Xrec = Xrec.reshape((Xrec.shape[0], -1))

xflat = X.reshape((X.shape[0], -1))
W = layers['hid'].W#p, h
R = layers['pre_output'].W#h, p

print(layers['pre_output'].W.get_value().shape)
A = get_deriv(hid_nonlinearity, xflat) #n, p (derivative of activation function)
J = T.tensordot(R[None, :, :] * A[:, None, :], W, axes=([1], [1]))
J = J * get_deriv(out_nonlinearity, Xrec, direct=True)[:, :, None]
#symmetricity mesure (see : http://arxiv.org/pdf/1506.07643v3.pdf)
def norm(a):
    if a.ndim == 3:
        axes = (1, 2)
    else:
        axes = tuple(range(a.ndim))
    return (a ** 2).sum(axis=axes)
def symmetricity(J):
    if J.ndim == 3:
        J_t = J.transpose((0, 2, 1))
    else:
        J_t = J.T
    return norm(0.5 * (J + J_t))/ norm(J)

#J = theano.gradient.jacobian(Xrec, X)
J = symmetricity(J)
J = J.reshape((J.shape[0],))
get_jacobian = theano.function([X], J)

In [ ]:
X = T.vector()
Xrec = lasagne.layers.get_output(layers['output'], X.reshape((1, c, w, h)))
Xrec = Xrec.flatten()
J = theano.gradient.jacobian(Xrec, X)
J = symmetricity(J)
get_jacobian = theano.function([X], J)

In [ ]:
m = np.dot(layers['pre_output'].W.get_value().T, layers['hid'].W.get_value().T)
print(symmetricity(m))

In [ ]:
from helpers import salt_and_pepper
def sample(x):
    return (np.random.uniform(size=x.shape) <= x).astype(np.float32)

In [ ]:
import numpy as np
seed = 42
#seed = np.random.randint(0, 99999)
#np.random.seed(seed)
jac = []
N = 10
s = np.random.uniform(size=(N, c, w, h))
s = s.astype(np.float32)
samples = []

samples.append(s.copy()[None, :, :, :, :])

#idx = np.random.randint(0, data.X.shape[0])
#s = data.X[idx:idx + 1].reshape((1, c, w, h))

print(s.shape)
nb_iter = 400
loss = []
for i in range(nb_iter - 1):
    sprev = s    
    #s = capsule.reconstruct(salt_and_pepper(s, corruption_level=0.3, backend='numpy'))
    s = capsule.reconstruct(s)
    s = s.astype(np.float32)
    samples.append(s.copy()[None, :, :, :, :])
    #samples = s.copy()
    #s = np.random.binomial(n=1, p=s, size=s.shape).astype('float32')# binarize by sampling
    #s = s > 0.5
    #s = sample(s)
    #j = get_jacobian(s.flatten())
    #jac.append(j)
    #print(j.mean())
    score = np.abs(s - sprev).sum()
    print(score)
    loss.append(score)
    #if score == 0:
    #    break
samples = np.concatenate(samples, axis=0)
print(samples.shape)
#samples = samples.transpose((1, 0, 2, 3, 4))
#print(samples.shape)
#samples = s
#print(samples.shape)

In [ ]:
print(samples.shape)

In [ ]:
from lasagnekit.misc.plot_weights import dispims_color, tile_raster_images
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.io import imsave
#plt.plot(loss)
print("Nb of iterations", len(loss))
fig = plt.figure(figsize=(20, 20))

sw, sh = samples.shape[0:2]
samples_ = samples.reshape((sw*sh, c, w, h))
print(samples_.shape)
if samples_.shape[1] == 1:
    img = tile_raster_images(samples_[:, 0], (w, h), (sw, sh), tile_spacing=(2, 2), )
    imsave('out.png', img)
    #plt.imshow(img, cmap="gray", interpolation='none')
    #plt.axis('off')
    #plt.show()
else:
    ss = s.transpose((0, 2, 3, 1))
    img = dispims_color(ss)
    plt.imshow(img)
    plt.axis('off')
    plt.show()